In [2]:
from typing import List, Mapping, Tuple
from datasets import load_dataset, DatasetDict, Dataset
from transformers import pipeline
import torch
from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
from transformers import (
    AutoTokenizer,
    BatchEncoding,
    AutoConfig, 
    AutoModelForQuestionAnswering,
    AutoModelForSeq2SeqLM,
    DataCollatorWithPadding,
    LayoutLMTokenizerFast,
    LayoutLMForTokenClassification,
    TrainingArguments,
    Trainer)
from datasets import load_dataset
import numpy as np
from pathlib import Path


2024-09-12 18:30:46.842255: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
nlp = pipeline(
    "document-question-answering",
    model="impira/layoutlm-invoices",
    device=device
)

Some weights of the model checkpoint at impira/layoutlm-invoices were not used when initializing LayoutLMForQuestionAnswering: ['token_classifier_head.bias', 'token_classifier_head.weight']
- This IS expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [43]:
nlp(
    "../data/train/preprocess/0.png",
    "What is the date of document generation?"
)

[{'score': 0.8136457204818726,
  'answer': '2024-05-07',
  'start': 191,
  'end': 191}]

In [7]:
nlp(
    "../data/train/preprocess/0.png",
    "What is the document number?"
)

[{'score': 0.9881201386451721, 'answer': 'BA-145085', 'start': 15, 'end': 15}]

In [12]:
from transformers import LayoutLMForQuestionAnswering

In [23]:
peft_config = LoraConfig(
                        r=32,
                        lora_alpha=32,
                        lora_dropout=0.05,
                        target_modules = ["qa_outputs"])

In [41]:
from transformers import AutoProcessor
processor = AutoProcessor.from_pretrained(model_name)

In [31]:
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)

In [20]:
model_name = "impira/layoutlm-invoices"
model = LayoutLMForQuestionAnswering.from_pretrained(model_name).to(device)

Some weights of the model checkpoint at impira/layoutlm-invoices were not used when initializing LayoutLMForQuestionAnswering: ['token_classifier_head.bias', 'token_classifier_head.weight']
- This IS expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LayoutLMForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
def tokenizer_func(example: str):
    text = example["text"]
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text, #We pass the text of the dataset
        return_tensors="pt", #The datatype that we want the output data
        truncation=True,
        padding=True,
        max_length=4065 #This is intrinsic to the len of the model
    )
    labels = tokenizer(example["output"],
        return_tensors="pt", #The datatype that we want the output data
        truncation=True,
        padding=True,
        max_length=4065 #This is intrinsic to the len of the model
    )
                       
    tokenized_inputs["labels"]= labels["input_ids"]
    return tokenized_inputs

In [33]:
if tokenizer.pad_token is None:
    #Se debe adicionar el token de pad
    tokenizer.add_special_tokens({"pad_token": "[PAD]"}) 
    #También se debe actualizar en el modelo los nuevos largos del token
    model.resize_token_embeddings(len(tokenizer))

Lets try to fine tune the model

In [16]:
model

LayoutLMForQuestionAnswering(
  (layoutlm): LayoutLMModel(
    (embeddings): LayoutLMEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768)
      (x_position_embeddings): Embedding(1024, 768)
      (y_position_embeddings): Embedding(1024, 768)
      (h_position_embeddings): Embedding(1024, 768)
      (w_position_embeddings): Embedding(1024, 768)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): LayoutLMEncoder(
      (layer): ModuleList(
        (0-11): 12 x LayoutLMLayer(
          (attention): LayoutLMAttention(
            (self): LayoutLMSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
   

In [27]:
model = get_peft_model(model, peft_config).to(device)
model.print_trainable_parameters()

trainable params: 24,640 || all params: 127,817,538 || trainable%: 0.0193


In [28]:
lr = 1e-3
batch_size = 4
num_epochs = 20

In [29]:
training_args = TrainingArguments(
    output_dir= "../models/" + model_name + "-lora-data-extraction",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    bf16=True
)

In [30]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_dataset["train"],
    eval_dataset=tokenizer_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator, # this will dynamically pad examples in each batch to be equal length
)

NameError: name 'tokenizer_dataset' is not defined

Initial conversion

In [80]:
from pdf2image import convert_from_path
from PIL import Image

In [6]:
pages = convert_from_path("../data/train/preprocess/0.pdf")

In [7]:
pages[0].save("../data/train/preprocess/0.png","PNG")

### convert the pdfs

In [89]:
import os
from natsort import natsorted
cont = 0
os.makedirs("../data/train/img", exist_ok=True)
data_path = Path("../data/")
pdf_files = [data_path / "train/preprocess" / f for f in os.listdir(data_path/ "train/preprocess") if f.endswith(".pdf")]
pdf_files = natsorted(pdf_files)
for f in pdf_files:
    try:
        pages = convert_from_path(f)
        pages[0].save(f"../data/train/img/{cont}.png", "PNG")
        cont +=1
    except Exception as e:
        print(f"file {cont} has password")

file 28 has password
file 71 has password


Whit Llava

In [16]:
from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch
from PIL import Image
import requests

processor = LlavaNextProcessor.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf")

model = LlavaNextForConditionalGeneration.from_pretrained("llava-hf/llava-v1.6-mistral-7b-hf", torch_dtype=torch.float16, low_cpu_mem_usage=True) 
model.to("cuda:0")

# prepare image and text prompt, using the appropriate prompt template
image = Image.open("../data/train/preprocess/0.png")




Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
What is the Total price? [/INST] The total price on the invoice is 165,490. 


In [23]:
# Define a chat histiry and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image") 
conversation = [
    {

      "role": "user",
      "content": [
          {"type": "text", "text": "what is the total value including tax?"},
          {"type": "image"},
        ],
    },
]
prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

inputs = processor(images=image, text=prompt, return_tensors="pt").to("cuda:0")

# autoregressively complete prompt
output = model.generate(**inputs, max_new_tokens=100)

print(processor.decode(output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST]  
what is the total value including tax? [/INST] The total value including tax is 165,460.00. 


From anotations

In [55]:
from uuid import uuid4
from PIL import Image
def convert_to_ls(
    image: Image.Image, 
    tesseract_output: Mapping, 
    default_label: str = "O"
):
    """
    :param image: PIL image object
    :param tesseract_output: the output from tesseract
    :param per_level: control the granularity of bboxes from tesseract
    :return: tasks.json ready to be imported into Label Studio with "Optical Character Recognition" template

    * https://labelstud.io/guide/predictions.html#Import-bbox-and-choice-pre-annotations-for-images
    * https://labelstud.io/blog/improve-ocr-quality-for-receipt-processing-with-tesseract-and-label-studio/#takeaways-for-ocr-with-tesseract-and-label-studio

    """
    image_width, image_height = image.size
    results = []
    all_scores = []
    for i, text in enumerate(tesseract_output["text"]):
        if text.strip() != "" and tesseract_output["conf"][i] != "-1":
            bbox = {
                "x": 100 * tesseract_output["left"][i] / image_width,
                "y": 100 * tesseract_output["top"][i] / image_height,
                "width": 100 * tesseract_output["width"][i] / image_width,
                "height": 100 * tesseract_output["height"][i] / image_height,
                "rotation": 0,
            }
            region_id = str(uuid4())[:10]
            score = tesseract_output["conf"][i]
            bbox_result = {
                "id": region_id,
                "from_name": "bbox",
                "to_name": "image",
                "type": "rectangle",
                "value": bbox,
            }
            transcription_result = {
                "id": region_id,
                "from_name": "transcription",
                "to_name": "image",
                "type": "textarea",
                "value": dict(text=[text], **bbox),
                "score": score,
            }
            label_result = {
                "id": region_id,
                "from_name": "label",
                "to_name": "image",
                "type": "labels",
                "value": dict(labels=[default_label], **bbox),
            }
            results.extend([bbox_result, label_result, transcription_result])
            all_scores.append(score)

    return{
# JSON Format as expected by Label Studio for importing pre-annotated OCR
        "data": {"ocr": image.filename},  # Presigned URL
        "predictions": [
            {
                "result": results,
                "score": sum(all_scores) / len(all_scores) if all_scores else 0,
            }
        ],
        }

In [53]:
from PIL import Image
from pathlib import Path
from typing import Iterator
from tqdm import tqdm

import pytesseract

def prepare_data_for_ls(images: Iterator[Image.Image]) -> List[Mapping]:
    """Parse image and extract words with bboxes
    Args:
        images (Image.Image): List of invoices converted in PIL images

    Return:
        (List[Mapping]): Label Studio JSON format for labeled OCR task
    """
    tasks = []
    for image in tqdm(images):
        tesseract_output = pytesseract.image_to_data(
            image, output_type=pytesseract.Output.DICT
        )
        task = convert_to_ls(image, tesseract_output)
        tasks.append(task)
    return tasks

In [50]:
from natsort import natsorted
data_path = Path("../data")
image_files = [data_path / "train/img" / f for f in os.listdir(data_path/ "train/img") if f.endswith(".png")]
image_files = natsorted(image_files)
image_data = [Image.open(f) for f in image_files]

In [56]:
prepared_data = prepare_data_for_ls(image_data)

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


...

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


100%|██████████| 97/97 [02:08<00:00,  1.33s/it]

dict_keys(['level', 'page_num', 'block_num', 'par_num', 'line_num', 'word_num', 'left', 'top', 'width', 'height', 'conf', 'text'])


In [61]:
import json
with open("../data/train/data_ocr.json", "w")as f:
    data = json.dump(prepared_data, f, indent=4)

In [3]:
import json
with open("../data/train/preprocess/project-2-at-2024-09-12-23-28-dcd5843d.json")as f:
    data = json.load(f)

In [19]:
def ls_to_dataset(
    ls_data: List[Mapping], test_size: float = 0.3
) -> DatasetDict:
    """Convert labeled documents into a DHF Dataset."""
    mapping = []
    for idx, doc in enumerate(ls_data):
        original_width = doc["bbox"][0]["original_width"]
        original_height = doc["bbox"][0]["original_height"]
        bboxes = [
            (
                bbox["x"], 
                bbox["y"], 
                bbox["x"] + bbox["width"], 
                bbox["y"] + bbox["height"]
            ) for bbox in doc["bbox"]
        ]
        bboxes = unormalize_bbox_from_ls(
            bboxes=bboxes,
            width=original_width,
            height=original_height
        )
        labels = [label["labels"][0] for label in doc["label"]]
        words = [word for word in doc["transcription"]]
        mapping.append(
            {
                "doc_id": idx,
                "bboxes": bboxes,
                "words": words,
                "labels": labels,
                "original_width": original_width,
                "original_height": original_height
            }
        )
    dataset_dict = Dataset.from_list(mapping).train_test_split(test_size=test_size, shuffle=True)
    #dataset_dict = Dataset.from_list(mapping)
    return dataset_dict

def unormalize_bbox_from_ls(
    bboxes: List[Tuple[int, int, int, int]], 
    width: int, 
    height: int
) -> List[List]:
    """To label documents, each bbox was normalized into 100*x0/W."""
    x = np.array(bboxes)
    v = (width/100, height/100, width/100, height/100)
    X = x*v
    return X.round(2).tolist()

In [20]:
dataset_dict = ls_to_dataset(data)

In [21]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['doc_id', 'bboxes', 'words', 'labels', 'original_width', 'original_height'],
        num_rows: 2
    })
    test: Dataset({
        features: ['doc_id', 'bboxes', 'words', 'labels', 'original_width', 'original_height'],
        num_rows: 2
    })
})

In [11]:
def preprocess(
    element: Mapping, # Batched
    tokenizer: AutoTokenizer, 
    labels_ref: List[str],
    inference_mode: bool = False
) -> Mapping:
    """https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__

    Args:
        element (Mapping): Batched data
        tokenizer (AutoTokenizer): Word tokenizer
        labels_ref (List[str]): List of labels:
            labels = [
                "Date",
                "Company_name",
                "NIT",
                "Num_bill",
                "Base",
                "IVA",
                "Total"
             ]
    """
    encoding: BatchEncoding = tokenizer(
        element["words"],
        truncation=True,
        return_tensors="pt",
        padding="max_length",      # Max_length of the model (here 512 for LayoutLM-base)
        is_split_into_words=True,  # Token classification
    )
    encoding["labels"] = align_labels(
        encoding=encoding, 
        batched_labels=element["labels"], 
        labels_ref=labels_ref
    )
    encoding["bbox"] = align_bboxes(
        encoding=encoding,
        batched_bboxes=element["bboxes"],
        widths=element["original_width"],
        heights=element["original_height"],
        max_tokens=tokenizer.model_max_length,
    )

    encoding = encoding.convert_to_tensors(tensor_type="pt") # For labels and bbox
    
    assert encoding["input_ids"].shape[-1] == tokenizer.model_max_length
    assert encoding["labels"].shape[-1] == tokenizer.model_max_length
    assert encoding["bbox"].shape[1] == tokenizer.model_max_length

    return encoding


def align_labels(
    encoding, 
    batched_labels: List[List[str]], 
    labels_ref: List[str]
) -> List[List[int]]:
    label2id = {k: v for v, k in enumerate(labels_ref)}

    batched_aligned_labels = []
    for i, labels in enumerate(batched_labels):
        label_ids = []
        word_ids = encoding.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label2id[labels[word_idx]])  # Transform label into it's respective ID
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        batched_aligned_labels.append(label_ids)
    return batched_aligned_labels


def align_bboxes(
    encoding,
    batched_bboxes,
    widths,
    heights,
    max_tokens,
    cls_token_box=[0, 0, 0, 0],
    sep_token_box=[1000, 1000, 1000, 1000],
    pad_token_box=[0, 0, 0, 0],
) -> List[List[List[int]]]:
    batched_aligned_bboxes = []
    for i, bboxes in enumerate(batched_bboxes):
        aligned_bboxes = []
        word_ids = encoding.word_ids(batch_index=i)  # Map tokens to their respective word.
        word_ids = [word_idx for word_idx in word_ids if word_idx is not None]  # Remove None corresponding to CLS and EOS
        for word_idx in word_ids:
            normalized_bbox = normalize_bbox(
                bbox=bboxes[word_idx], width=widths[i], height=heights[i]
            )
            aligned_bboxes.append(normalized_bbox)
        aligned_bboxes = [cls_token_box] + aligned_bboxes + [sep_token_box] # Add bboxes for CLS and EOS
        aligned_bboxes += [pad_token_box] * (max_tokens - len(aligned_bboxes)) # Padding
        aligned_bboxes = aligned_bboxes[:max_tokens]
        batched_aligned_bboxes.append(aligned_bboxes)
    return batched_aligned_bboxes


def normalize_bbox(bbox, width: int, height: int) -> List[int]:
    return [
        int(1000 * (bbox[0] / width)),
        int(1000 * (bbox[1] / height)),
        int(1000 * (bbox[2] / width)),
        int(1000 * (bbox[3] / height)),
    ]

In [28]:
def preprocess_dataset(
    dataset: DatasetDict, tokenizer: AutoTokenizer, labels_ref: List[str]
) -> DatasetDict:
    """
    Return:
      Features: ['input_ids', 'token_type_ids', 'attention_mask', 'bbox', "labels"])
    """
    processed_dataset = dataset.map(
        preprocess,
        fn_kwargs={"tokenizer": tokenizer, "labels_ref": labels_ref},
        remove_columns=dataset["train"].column_names,
        keep_in_memory=False,
        batched=True
    )

    return processed_dataset

In [14]:
labels = [
                "Date",
                "Company_Name",
                "Nit",
                "Num_bill",
                "Base",
                "IVA",
                "Total"
             ]
id2label = {v: k for v, k in enumerate(labels)}
label2id = {k: v for v, k in enumerate(labels)}

model_name = "microsoft/layoutlm-base-uncased"
tokenizer = LayoutLMTokenizerFast.from_pretrained(model_name)
model = LayoutLMForTokenClassification.from_pretrained(
    "microsoft/layoutlm-base-uncased",
    num_labels=len(labels), 
    label2id=label2id, 
    id2label=id2label
).to(device)

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
dataset_dict_preprocess = preprocess_dataset(dataset_dict, tokenizer, labels)

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [31]:
dataset_dict_preprocess

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask', 'bbox'],
        num_rows: 2
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask', 'bbox'],
        num_rows: 2
    })
})

In [32]:
lr = 1e-3
batch_size = 4
num_epochs = 20

In [33]:
training_args = TrainingArguments(
    output_dir= "../models/" + model_name + "-fine-tuned",
    num_train_epochs=num_epochs,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    warmup_steps=10,
    learning_rate=5e-5,
    # logging & evaluation strategies
)

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict_preprocess["train"],
    eval_dataset=dataset_dict_preprocess["test"]
)

trainer.train()

  0%|          | 0/20 [00:00<?, ?it/s]

{'train_runtime': 2.9767, 'train_samples_per_second': 13.438, 'train_steps_per_second': 6.719, 'train_loss': 1.162592315673828, 'epoch': 20.0}


TrainOutput(global_step=20, training_loss=1.162592315673828, metrics={'train_runtime': 2.9767, 'train_samples_per_second': 13.438, 'train_steps_per_second': 6.719, 'total_flos': 10524914688000.0, 'train_loss': 1.162592315673828, 'epoch': 20.0})